# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
spaceship.dropna(inplace = True)

# Feature Scaling

**Perform Train Test Split**

In [4]:
features = spaceship[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]
target = spaceship['Transported']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

**Feature Scaling**

In [5]:
# We initialize the MinMayScaler instance.
normalizer = MinMaxScaler() 

# We fit our model to the feautures of the training set, while keeping in mind to only fit it to the training data in order to prevent data leakage.
normalizer.fit(X_train)

# We apply the normalizer function to the train and test feature data.
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

# We cast our normalized training and test data into a new dataframe and display.
X_train = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [6]:
X_train.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.405063,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.050633,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.379747,0.00000,0.007916,0.000000,0.051276,0.000000
3,0.215190,0.00131,0.000000,0.046111,0.016378,0.000049
4,0.329114,0.00000,0.000000,0.000000,0.000000,0.000000


In [7]:
X_test.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.632911,0.0,0.000000,0.0,0.00000,0.0
1,0.227848,0.0,0.000000,0.0,0.00000,0.0
2,0.189873,0.0,0.000000,0.0,0.00000,0.0
3,0.658228,0.0,0.000000,0.0,0.00000,0.0
4,0.784810,0.0,0.054775,0.0,0.07774,0.0


## KNN Algorithm

In [8]:
# We initialize a new knn classifier instance without defining the hypterparameter yet.
knn = KNeighborsClassifier()

# We train the algorithm with our normalized training data.
knn.fit(X_train, y_train)

# We predict the class of our normalized test data
y_pred_knn = knn.predict(X_test)

In [9]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_knn)}')
print(f"Precision = {precision_score(y_test, y_pred_knn, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_knn, average='binary')}")

Accuracy = 0.7518910741301059
Precision = 0.7205298013245033
Recall = 0.8229954614220878


## Logistic Regression

In [10]:
epsilon = 1e-6
X_train_log = np.log(X_train_norm + epsilon)
X_test_log = np.log(X_test_norm + epsilon)

log = LogisticRegression()

log.fit(X_train_log, y_train)

y_pred_log = log.predict(X_test_log)

In [11]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_log)}')
print(f"Precision = {precision_score(y_test, y_pred_log, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_log, average='binary')}")

Accuracy = 0.7299546142208775
Precision = 0.7222222222222222
Recall = 0.7473524962178517


## Decision Tree

In [12]:
tree = DecisionTreeClassifier(max_depth=10)

tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test_norm)

/Users/johannapfeil/opt/anaconda3/envs/base_new/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [13]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_tree)}')
print(f"Precision = {precision_score(y_test, y_pred_tree, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_tree, average='binary')}")

Accuracy = 0.745839636913767
Precision = 0.7163781624500666
Recall = 0.8139183055975794


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

KNN seems to be the best model so far, so we will try ensemble methods on it.

- **Bagging and Pasting**

In [14]:
bagging = BaggingClassifier(KNeighborsClassifier(), n_estimators=100, max_samples = 1000)

bagging.fit(X_train, y_train)

y_pred_bag = bagging.predict(X_test)

In [15]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_bag)}')
print(f"Precision = {precision_score(y_test, y_pred_bag, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_bag, average='binary')}")

Accuracy = 0.75642965204236
Precision = 0.7116104868913857
Recall = 0.8623298033282905


- **Random Forests**

In [16]:
forest = RandomForestClassifier(n_estimators = 100, max_depth = 20)
forest.fit(X_train, y_train)
y_pred_forest = forest.predict(X_test)

In [17]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_forest)}')
print(f"Precision = {precision_score(y_test, y_pred_forest, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_forest, average='binary')}")

Accuracy = 0.7738275340393344
Precision = 0.7465940054495913
Recall = 0.8290468986384266


- **Gradient Boosting**

In [18]:
gb = GradientBoostingClassifier(max_depth=20, n_estimators=100)
gb.fit(X_train, y_train)
y_pred_gb = forest.predict(X_test)

In [19]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_gb)}')
print(f"Precision = {precision_score(y_test, y_pred_gb, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_gb, average='binary')}")

Accuracy = 0.7738275340393344
Precision = 0.7465940054495913
Recall = 0.8290468986384266


- **Adaptive Boosting**

In [20]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=100)
ada.fit(X_train, y_train)
y_pred_ada = ada.predict(X_test)

In [21]:
print(f'Accuracy = {accuracy_score(y_test, y_pred_ada)}')
print(f"Precision = {precision_score(y_test, y_pred_ada, average='binary')}")
print(f"Recall = {recall_score(y_test, y_pred_ada, average='binary')}")

Accuracy = 0.7428139183055976
Precision = 0.7125827814569536
Recall = 0.8139183055975794


Which model is the best and why?

All of them have their advantages and disadvantages. Adaptive boosting performs the worst. Bagging and Pasting performs better in